In [1]:
import requests
from proxy_requests import ProxyRequests
from bs4 import BeautifulSoup
import threading
import csv
import re

%config IPCompleter.greedy=True

global tot_users
global results
results = {True}

In [2]:
# takes url of the user search results
# ex: https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1
def url_to_soup(url, proxy = False):
    url_prefix = 'https://'
    if (url_prefix not in url):
        url = url_prefix + url
    
    r = ""
    isRetry = False
    while (str(r) == ""):
        if (isRetry):
            print('Thread:', threading.currentThread().getName())
            print("Retrying Bad Request")
            
        if (proxy):
            r = ProxyRequests(url)
            r.get()
        else:
            r = requests.get(url)
            
        isRetry = True

    return BeautifulSoup(str(r), 'html.parser')

def soup_to_users(soup):
    # content > table > tbody > div > a
    users = soup.find_all('a', {"href" : re.compile(r"/profile/*")})
    # filters all users
    users = list(filter(lambda elem : elem.get_text() != "", users))
    users = [elem.get_text() for elem in users]
    
    return users

# hard coded approach that allows multi-threading
def base_url_to_pg_link(url, pg):
    return (url + str(24 * pg))

# useless for multi-threading
def url_and_soup_to_pg_link(url, soup, pg):
    # takes domain of url
    url = url.replace("https://","")
    url = url.replace("www.", "")
    url = url.split('/')[0]
    
    # think about optimizing this line so it doesn't have to iterate twice
    pg_links = soup.find_all('a', {"href" : re.compile(r"/users.php*")})
    url_path = [link['href'] for link in pg_links 
                if link.get_text() == str(pg)][0]
    
    return (url + url_path)

# relies on a valid list called results
def store_scrape_results(func):
    def wrapper(*args, **kwargs):
        results.add(func(*args, **kwargs))
    
    return wrapper

# scrapes and inputs users
# returns false when no users left
# pg starts index at 0
@store_scrape_results
def scrape_users(user_list, base_url, pg, is_proxy = False):
    url = base_url_to_pg_link(base_url, pg)
    soup = url_to_soup(url, proxy = is_proxy)
    users = soup_to_users(soup)
    if (len(users) == 0):
        return False
    user_list.extend(users)
    return True

def scrape_users_in_gender(base_url, fn, pg_cap = 30000, cap_threads = 32):
    tot_users = list()
    results = {True}
    pg_num = 0

    print("Start Scraping")
    start_threads = threading.activeCount()
    try:
        while (pg_num < pg_cap and False not in results):
            # have cap_threads number of threads running at a time to complete task
            while ((threading.activeCount() - start_threads) < cap_threads and 
                   False not in results):

                t = threading.Thread(
                    target = scrape_users,
                    kwargs = {
                        'user_list' : tot_users, 
                        'base_url' : base_url, 
                        'pg' : pg_num, 
                        'is_proxy' : True,
                    }
                )
                t.start()

                pg_num += 1
                
                if (pg_num % 100 == 0):
                    print('Current Number of Threads:', (threading.activeCount() - start_threads))
                    print("Number of Users:", len(tot_users))
                    print("Current Page:", pg_num)
                    print("Results so Far:", results)

        while (threading.activeCount() != start_threads):
            pass
        
    except (KeyboardInterrupt, SystemExit):
        csv_write(tot_users, fn = 'temp_users.csv')
    
    print("End Scraping")
    print("Inputting Users...")
    csv_write(tot_users, fn)
    print("Finished")

def csv_write(users, fn):
    with open(fn, 'w', newline = '') as open_test:
        my_csv = csv.writer(open_test)
        for user in users:
            my_csv.writerow([user])
            
# combines 2 csv files into 1
def combine_csv(fn1, fn2, comb_fn):
    with open(fn1, 'r') as f:
        reader = csv.reader(f)
        my_set = list(reader)
        my_set = [user_list[0] for user_list in my_set]
        my_set = set(my_set)

    with open(fn2, 'r') as f:
        reader = csv.reader(f)
        my_set2 = list(reader)
        my_set2 = [user_list[0] for user_list in my_set2]
        my_set2 = set(my_set2)
        
    user_list = list(my_set | my_set2)
    csv_write(user_list, comb_fn)

In [3]:
f_base_url = 'https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show='

scrape_users_in_gender(f_base_url, fn = 'f_users.csv')

Start Scraping
Current Number of Threads: 32
Number of Users: 0
Current Page: 100
Results so Far: {True}
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=2592
Thread: Thread-114
Retrying Bad Request
Current Number of Threads: 32
Number of Users: 24
Current Page: 200
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 48
Current Page: 300
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 48
Current Page: 400
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 96
Current Page: 500
Results so Far: {True}
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=10776
Thread: Thread-455
Retrying Bad Request
Current Number of Threads: 32
Number of Users: 96
Current Page: 600
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 120
Current Page: 700
Results so Far

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=36696
Thread: Thread-1535
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=35664
Thread: Thread-1492
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=36792
Thread: Thread-1539
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=35952
Thread: Thread-1504
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=36888
Thread: Thread-1543
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=39792
Thread: Thread-1664
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=39024
Thread: Thread-1632
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=39816
Thread: Thread-1665
Retrying Bad Request
Current Number of Threads: 32
Number of Users: 8784
Current Page: 1700
Results so Far: {True}
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=37488
Thread: Thread-1568
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=38520
Thread: Thread-1611
Retrying Bad Request
Unable

Current Number of Threads: 32
Number of Users: 12360
Current Page: 1900
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 12888
Current Page: 2000
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 13128
Current Page: 2100
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 13224
Current Page: 2200
Results so Far: {True}
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=51456
Thread: Thread-2150
Retrying Bad Request
Current Number of Threads: 32
Number of Users: 13368
Current Page: 2300
Results so Far: {True}
Current Number of Threads: 31
Number of Users: 13464
Current Page: 2400
Results so Far: {True}
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=55224
Thread: Thread-2307
Retrying Bad Request
Current Number of Threads: 31
Number of Users: 13464
Current Page: 250

Current Number of Threads: 32
Number of Users: 21384
Current Page: 8300
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 22152
Current Page: 8400
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 22944
Current Page: 8500
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 23448
Current Page: 8600
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 24576
Current Page: 8700
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 25344
Current Page: 8800
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 25800
Current Page: 8900
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 26376
Current Page: 9000
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 26856
Current Page: 9100
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 27456
Current Page: 9200
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 280

...

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=358488
Thread: Thread-14943
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=357360
Thread: Thread-14896
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=357768
Thread: Thread-14913
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=357720
Thread: Thread-14911
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=357840
Thread: Thread-14916
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use


Thread: Thread-14984
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=360096
Thread: Thread-15010
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=359592
Thread: Thread-14989
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=360072
Thread: Thread-15009
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=360336
Thread: Thread-15020
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=360264
Thread: Thread-15017
Retrying Bad Request
Unable to make proxied request... Please check 

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=360504Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=360864
Thread: Thread-15042

Thread: Thread-15027
Retrying Bad Request
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=361320
Thread: Thread-15061
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=361368
Thread: Thread-15063
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=360936
Thread: Thread-15045
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Current Number of Threads: 32
Number of Users: 127248
Current Page: 17600
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 17700
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 17800
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 17900
Results so Far: {True}
Current Number of Threads: 31
Number of Users: 127248
Current Page: 18000
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 18100
Results so Far: {True}
Current Number of Threads: 31
Number of Users: 127248
Current Page: 18200
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 18300
Results so Far: {True}
Current Number of Threads: 31
Number of Users: 127248
Current Page: 18400
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 18500
Results so Far: {True}
Current Number of Threads: 32


Current Number of Threads: 32
Number of Users: 127248
Current Page: 25900
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 26000
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 26100
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 26200
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 26300
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 26400
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 127248
Current Page: 26500
Results so Far: {True}
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=632352
Thread: Thread-26354
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show=632616

In [4]:
nb_base_url = 'https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show='

scrape_users_in_gender(nb_base_url, fn = 'nb_users.csv')

Start Scraping
Current Number of Threads: 31
Number of Users: 792
Current Page: 100
Results so Far: {True}
Current Number of Threads: 30
Number of Users: 1440
Current Page: 200
Results so Far: {True}
Current Number of Threads: 30
Number of Users: 1608
Current Page: 300
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 400
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 500
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 600
Results so Far: {True}
Current Number of Threads: 30
Number of Users: 1608
Current Page: 700
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 800
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 900
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 1000
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Cur

Current Number of Threads: 32
Number of Users: 1608
Current Page: 8500
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 8600
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 8700
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 8800
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 8900
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 9000
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 9100
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 9200
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 9300
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current Page: 9400
Results so Far: {True}
Current Number of Threads: 32
Number of Users: 1608
Current 

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=335256
Thread: Thread-43976
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=335352
Thread: Thread-43980
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=335592
Thread: Thread-43990
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=335688
Thread: Thread-43994
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=335760
Thread: Thread-43997
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=337992
Thread: Thread-44090
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=338136
Thread: Thread-44096
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=338280
Thread: Thread-44102
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=336696
Thread: Thread-44036
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=337728
Thread: Thread-44079
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=339144
Thread: Thread-44138
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=339072
Thread: Thread-44135
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=339600
Thread: Thread-44157
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=339624
Thread: Thread-44158
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=339696
Thread: Thread-44161
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

...

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=701712
Thread: Thread-59245
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=702480
Thread: Thread-59277
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=701880
Thread: Thread-59252
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=701904
Thread: Thread-59253
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=702000
Thread: Thread-59257
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=705288
Thread: Thread-59394
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=705336
Thread: Thread-59396
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=705480
Thread: Thread-59402
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=704088
Thread: Thread-59344
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=705432
Thread: Thread-59400
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=708672
Thread: Thread-59535
Retrying Bad Request
Current Number of Threads: 32
Number of Users: 1608
Current Page: 29600
Results so Far: {True}
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=709008
Thread: Thread-59549
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=709056
Thread: Thread-59551
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=709176
Thread: Thread-59556
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=708504
Thread: Thread-59528
Retrying Bad Req

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=712248
Thread: Thread-59684
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=710856
Thread: Thread-59626
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=712320
Thread: Thread-59687
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=712368
Thread: Thread-59689
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=709872
Thread: Thread-59585
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=713784
Thread: Thread-59748
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=714792
Thread: Thread-59790
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=714816
Thread: Thread-59791
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=713808
Thread: Thread-59749
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=3&show=714768
Thread: Thread-59789
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

In [3]:
m_base_url = 'https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show='
f_base_url = 'https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=2&show='

scrape_users_in_gender(m_base_url, fn = 'm_users.csv')

Start Scraping
Current Page: 100
Current Page: 200
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=96
Thread: Thread-10
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=672
Thread: Thread-34
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=648
Thread: Thread-33
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=480
Thread: Thread-26
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=360
Thread: Thread-21
Retrying Bad Request
Unable to make proxied request... Please check the validity of https:/

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=12024
Thread: Thread-507
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=11376
Thread: Thread-480
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=8544
Thread: Thread-362
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=11544
Thread: Thread-487
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=11400
Thread: Thread-481
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&a

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=13872
Thread: Thread-584
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=13392
Thread: Thread-564
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=11952
Thread: Thread-504
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=13920
Thread: Thread-586
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=13968
Thread: Thread-588
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=14616
Thread: Thread-615
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=14640
Thread: Thread-616
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=14664
Thread: Thread-617
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=13872
Thread: Thread-584
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=14280
Thread: Thread-601
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=15624
Thread: Thread-657
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=15648
Thread: Thread-658
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=13392
Thread: Thread-564
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=14688
Thread: Thread-618
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=15072
Thread: Thread-634
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&

...

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=699888
Thread: Thread-29168
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=700176
Thread: Thread-29180
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=701064
Thread: Thread-29217
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=701112
Thread: Thread-29219
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=699384
Thread: Thread-29147
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=703128
Thread: Thread-29303
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=702576
Thread: Thread-29280
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=703344
Thread: Thread-29312
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=703368
Thread: Thread-29313
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=702096
Thread: Thread-29260
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=706104
Thread: Thread-29427
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=705408
Thread: Thread-29398
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=705432
Thread: Thread-29399
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=706344
Thread: Thread-29437
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=705552
Thread: Thread-29404
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=708432
Thread: Thread-29524
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=708552
Thread: Thread-29529
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=708600
Thread: Thread-29531
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=708648
Thread: Thread-29533
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=708792
Thread: Thread-29539
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/use

Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=716832
Thread: Thread-29874
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=717408
Thread: Thread-29898
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=717720
Thread: Thread-29911
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=717912
Thread: Thread-29919
Retrying Bad Request
Unable to make proxied request... Please check the validity of https://myanimelist.net/users.php?q=&loc=&agelow=0&agehigh=0&g=1&show=718128
Thread: Thread-29928
Retrying Bad Request
Current Page: 30000
Number of Users: 327672
Current Last 100 Users: ['-Shaka-', 'Rias_AMVs

In [ ]:
comb